## Final Project Submission

Please fill out:
* Student name: Bryan Jamieson
* Student pace: Full Time
* Scheduled project review date/time: 09/20/2019
* Instructor name: Victor Geislinger
* Blog post URL: 


# Flip or Flop

##### This exploritory data analysis of Kings County Housing Data is intended for house flippers, trying to get the most bang for their buck. With house flippers in mind, I was interested to see if there was a relationship between houses price, house age, and zipcode.

##### To figure this out, I used various data analysis and vizualisation techniques to understand the type of relationship between house price, house age, and zipcode.

## Objectives

1) Explore and understand the data
   - Look at trends, extreme, null and missing values.
   - Added column to Data Set called "house_age"
   
2) Clean & Visualize The Data 
   - Before looking at my main variables: zipcodes, price, and house age,
     I am going to clean my data by dealing with the missing values
   - I see outliers, which I am going to tackle later on.
   - I am going to look at the missing values again and then replace them.
 
3) Modeling
   - sklearn modeling
   - splitting dataset into train and test sets
   - statsmodeling
   - ols

# Data
#### The Kings County Housing Data Set is named "kc_house_data.csv". 

Import Modules, import and rename csv file

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
import matplotlib.pyplot as plt
import scipy.stats as stats
from matplotlib import rcParams

%matplotlib inline
%pylab inline 

kc_df = pd.read_csv('kc_house_data.csv')

# 1.) Explore and understand the data

###### * Look at trends, extreme, null and missing values
###### * Added column to Data Set called "house_age"

In [ ]:
kc_df.head()

# Take a peek at the data and see if anything does not look right off the start.
# I can already see a NaN value for waterfront and year renovated aswell.

###### * Exploring column names and index range to see if there are missing values in the dataset.
###### * Also noting the type of data: integer, float, or object. 

In [ ]:
kc_df.info()
# 21 columns, 21597 data entries

###### * My code above returned that I have missing values in "waterfront", "view", and "year renovated"
###### * Now I want to double check this using "isna()" 

In [ ]:
kc_df.isna().sum()

###### * The outputs above look like both methods pointed out the same number of missing values in the dataset.

#### Creating new variable "house_age" and adding it to our dataset.


In [ ]:
kc_df['house_age'] = 2019 - kc_df['yr_built']

In [ ]:
kc_df.head()

In [ ]:
kc_df['house_age']

In [ ]:
kc_df['house_age'].head()

# 2.) Clean & Visualize The Data

##### * Before looking at my main variables: zipcodes, price, and house age,
##### * I am going to clean my data by dealing with the missing values
##### * I see outliers, which I am going to tackle later on.
##### * I am going to look at the missing values again and then replace them.

######  Now I am going to copy the kc_df to look at the data - called "newkc_df"

In [ ]:
newkc_df = kc_df.copy()
newkc_df.head()

#Reference - https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.copy.html

###### Removing objects ('date', 'sqft_basement') and 'id' column,
###### because this data is not neccessary for what I am looking at.

In [ ]:
newkc_df.drop(columns = ['id', 'date', 'sqft_basement' ], inplace=True)

In [ ]:
newkc_df.info()

#### Checking to see what the column variables look like based on price.

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=6, figsize=(16,3))
for xcol, ax in zip(list(newkc_df)[1:7], axes):
                    newkc_df.plot(kind='scatter', x=xcol, y='price', ax=ax, alpha = 0.4, color = 'r');

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=6, figsize=(16,3))
for xcol, ax in zip(list(newkc_df)[8:14], axes):
                    newkc_df.plot(kind='scatter', x=xcol, y='price', ax=ax, alpha = 0.4, color = 'r');

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=6, figsize=(16,3))
for xcol, ax in zip(list(newkc_df)[15:20], axes):
                    newkc_df.plot(kind='scatter', x=xcol, y='price', ax=ax, alpha = 0.4, color = 'r');

In [ ]:
newkc_df.isna().sum()

In [ ]:
newkc_df['yr_renovated'] = newkc_df['yr_renovated'].fillna(0)
newkc_df['view'] = newkc_df['view'].fillna(0)
newkc_df['waterfront'] = newkc_df['waterfront'].fillna(0)

newkc_df.isna().sum()

#https://datatofish.com/replace-nan-values-with-zeros/

In [ ]:
newkc_df.info()

#### Now that I dealt with missing values, I can use IQR to remove outliers from the dataframe.

In [ ]:
Q1 = newkc_df.quantile(0.25)
Q3 = newkc_df.quantile(0.75)
IQR = Q3 - Q1
print(IQR)

#https://towardsdatascience.com/ways-to-detect-and-remove-the-outliers-404d16608dba

In [ ]:
print(newkc_df < (Q1 - 1.5 * IQR)) |(newkc_df > (Q3 + 1.5 * IQR))

In [ ]:
newkc_df_out = newkc_df[~((newkc_df < (Q1 - 1.5 * IQR)) |(newkc_df > (Q3 + 1.5 * IQR))).any(axis=1)]
newkc_df_out.shape

#### * Now the outliers are removed from the Dataset "newkc_df"

##### Here we are trying to see the average house price for every house age (1, 2, 3, 4 (years old), etc.) 
##### Now I wan't to visualize the data and graph it with a bar graph.

## Question: Which zip codes can I get the best bang for my buck when purchasing a house?

In [ ]:
sns.set(rc={'figure.figsize': (16,12)})
new_df = kc_df[['house_age', 'price']].copy().rename(columns = {'median': 'median price $'})
new_df.groupby("house_age").median().plot(y='price', kind = 'bar')

###### Great, we can see that houses 76 years old have the lowest average price of around 290k.
###### This is the only average price based on house age under 300k. That's a good start

In [ ]:
newkc_df.info()

#### Now I want to visually look at how Price effects zipcode, and how house age effects zip code.
#### First I want to get an idea of my zipcode data.

In [ ]:
newkc_df['zipcode'].value_counts()

In [ ]:
newkc_df['zipcode'].describe()

## Question: Which zipcodes have the highest average house prices?
#### * Looking at how price effects zipcode

In [ ]:
red = dict(markerfacecolor='r')
sns.set(rc={'figure.figsize': (16,12)})
newkc_df.boxplot(by='zipcode', column='price', fontsize = 12, rot = 90, flierprops = red);

plt.title('Boxplots of Price Variation ($) Grouped by Zipcode', fontsize = 16)

plt.xlabel('King County Zipcodes', fontsize = 12);
plt.ylabel('Price, $', fontsize = 12);
plt.savefig("Price vs Zip.png")


# Reference: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.boxplot.html

## Question: Which zip codes have the oldest houses that I could renovate?
#### * Looking at how house_age effects zipcode


In [ ]:
green = dict(markerfacecolor='g')
sns.set(rc={'figure.figsize': (16,12)})
newkc_df.boxplot(by='zipcode', column='house_age', fontsize = 12, rot = 90, flierprops = green);

plt.title('Boxplots of House Age Variation Grouped by Zipcode', fontsize = 16)

plt.xlabel('King County Zipcodes', fontsize = 12);
plt.ylabel('Age', fontsize = 12);
plt.savefig("House Age vs Zip.png")


# Reference: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.boxplot.html

###### 98102 Zipcode average house age is between 90-95 years old. Yet, on average, the house prices range from 50k-100k.

In [ ]:
newkc_df.columns

In [ ]:
with sns.plotting_context("notebook",font_scale=2.5):
    g = sns.pairplot(newkc_df[['price','house_age','zipcode','sqft_living','bedrooms']], 
                 hue='bedrooms', palette='tab20',height=6)
g.set(xticklabels=[]);

# 3.) Modeling

## * sklearn modeling
#### * splitting the dataset into train and validation sets

In [ ]:
X = newkc_df[['house_age','zipcode','sqft_living','bedrooms']]
y = newkc_df[['price']]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 1/3, random_state = 10)

In [ ]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()

regressor.fit(X_train, y_train)

y_pred = regressor.predict(X_test)

from sklearn.metrics import r2_score, mean_squared_error

print(f"R2 score: {r2_score(y_test, y_pred)}")

print(f"MSE score: {mean_squared_error(y_test, y_pred)}")

### r2 score = 0.5503993021431164

In [ ]:
#r2 close to 1 is good, mse close to 0 is good.

In [ ]:
X = newkc_df[['house_age','zipcode','sqft_living','bedrooms', 'grade']]
y = newkc_df[['price']]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 1/3, random_state = 10)

In [ ]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()

regressor.fit(X_train, y_train)

y_pred = regressor.predict(X_test)

from sklearn.metrics import r2_score, mean_squared_error

print(f"R2 score: {r2_score(y_test, y_pred)}")

print(f"MSE score: {mean_squared_error(y_test, y_pred)}")

### r2 score = 0.6130299142907581

In [ ]:
#https://www.kaggle.com/divan0/multiple-linear-regression
#https://medium.com/data-science-everywhere/linear-regression-in-python-28dbe9674e36

## * statsmodels modeling

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [ ]:
m = ols('price ~ house_age',newkc_df).fit()
print (m.summary())

In [ ]:
sns.jointplot(x="house_age", y="price", data=newkc_df, kind = 'reg',fit_reg= True, height = 7)
plt.show()

###### Good to see graphically because it doesent make much sense
###### Since the price values and house age values are in completely different ranges.

In [ ]:
m = ols('price ~ house_age + zipcode',newkc_df).fit()
print (m.summary())

### r2 = .004 (WOW)

In [ ]:
m = ols('price ~ house_age + zipcode + sqft_living + bedrooms',newkc_df).fit()
print (m.summary())

#https://www.springboard.com/blog/data-mining-python-tutorial/

### r2 = .541 (WAHOO!)
### p value = 0 (WAHOOx2)

### With an adjusted r2 of .541, the relationship is moderately strong and we did a good job on our model.